# JurisCape Node B: The Audio Agent (Whisper)
**Role**: High-fidelity transcription of audio evidence.
**Model**: `openai/whisper-medium` (Switch to `large-v3` if Pro GPU available).

In [ ]:
# 1. Install Dependencies
!pip install fastapi uvicorn pyngrok python-multipart nest_asyncio requests openai-whisper
!apt-get install ffmpeg -y

In [ ]:
# 2. Load Whisper Model (This takes a moment)
import whisper
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Loading Whisper on {DEVICE}...")

# Using 'medium' fits comfortably on T4. Use 'large-v3' for max accuracy if VRAM allows.
model = whisper.load_model("medium", device=DEVICE)
print("Whisper Loaded!")

In [ ]:
# 3. Configure Ngrok
from pyngrok import ngrok
NGROK_TOKEN = "YOUR_NGROK_TOKEN_HERE"  # <--- PASTE TOKEN
ngrok.set_auth_token(NGROK_TOKEN)

In [ ]:
# 4. Start Server
import uvicorn
import os
from fastapi import FastAPI, Request, UploadFile, File
from fastapi.responses import JSONResponse
from pydantic import BaseModel
import shutil

SWARM_SECRET = "change-me-in-prod-secure-swarm-key"
app = FastAPI()

@app.middleware("http")
async def verify_secret(request: Request, call_next):
    if request.url.path in ["/docs", "/openapi.json"]:
        return await call_next(request)
    if request.headers.get("X-Swarm-Secret") != SWARM_SECRET:
        return JSONResponse(status_code=403, content={"detail": "Unauthorized"})
    return await call_next(request)

class AudioRequest(BaseModel):
    file_url: str

@app.post("/analyze_audio")
async def analyze_audio(payload: AudioRequest):
    print(f"Downloading audio from {payload.file_url}...")
    
    # Download the file (Mocking download for now, assuming direct link)
    # In prod: Use requests.get(payload.file_url) and save to temp
    import requests
    local_filename = "temp_audio.mp3"
    
    # MOCK: If url is 'test', we create a dummy file. Real implementation needs real URL.
    if payload.file_url == "test":
        # Create a dummy silent mp3 or handle error. 
        # For this demo, we assume the user provides a real downloadable URL or we skip download
        return {"transcript": "[TEST] This is a test transcript from the real node logic."}

    try:
        response = requests.get(payload.file_url, stream=True)
        if response.status_code == 200:
            with open(local_filename, 'wb') as f:
                shutil.copyfileobj(response.raw, f)
        else:
             return JSONResponse(status_code=400, content={"detail": "Could not download file"})

        # Run Inference
        print("Transcribing...")
        result = model.transcribe(local_filename)
        
        return {
            "transcript": result["text"],
            "language": result["language"],
            "segments": result["segments"] # Detailed timestamps
        }
    except Exception as e:
        return JSONResponse(status_code=500, content={"detail": str(e)})

# CLEANUP & RUN
ngrok.kill()
os.system("pkill ngrok")
tunnel = ngrok.connect(8000)
print(f"\n=== PUBLIC URL: {tunnel.public_url} ===\n")
config = uvicorn.Config(app, port=8000)
await uvicorn.Server(config).serve()